In [46]:

from torch.utils.data import DataLoader,TensorDataset
import torch.nn as nn
#import data_download
import functions
#import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import torch
import polars as pl
from torch.nn import functional as F
import torch.optim as optim

device = torch.device("cuda:0")


In [47]:
dbname='ai_dataset'
dbuser='postgres'
dbpassword='parola'
dbport=5432
dbhost='127.0.0.1'
table_name='tabela'

In [48]:
#conn.close()

In [49]:
tickers=["AAPL","MSFT",'SPY','XAUUSD.OANDA','BCO.ICMTRADER']
for i in range(len(tickers)):
    tickers[i]=tickers[i].replace('.','_')
net_functions=functions.Protected_execution

In [50]:
query=net_functions.create_query(tickers[0],table_name)
cursor,conn=net_functions.create_cursor(dbname,dbuser,dbpassword,dbport,dbhost)
data=pl.read_database(query=query,connection=conn)
data=data.with_columns(pl.col('aapl_adj_close').pct_change().alias('labels'))
data= data.select((pl.all()-pl.all().mean()) / pl.all().std())

data=data.drop_nulls()
labels=data['labels']

data


aapl_open,aapl_high,aapl_low,aapl_close,aapl_adj_close,aapl_volume,labels
f64,f64,f64,f64,f64,f64,f64
-0.830234,-0.829973,-0.82998,-0.805067,-0.805067,0.60523,0.087006
-0.830432,-0.830679,-0.830481,-0.805398,-0.805398,0.1657,-0.448443
-0.830155,-0.829503,-0.829947,-0.804325,-0.804325,0.824383,1.216513
-0.829176,-0.829178,-0.829152,-0.804463,-0.804463,0.748408,-0.218175
-0.829495,-0.826114,-0.829092,-0.801774,-0.801774,4.893222,3.063192
…,…,…,…,…,…,…
2.730181,2.694878,2.726402,2.712595,2.712595,-0.899152,-0.332392
2.701143,2.667074,2.723786,2.702483,2.702483,-0.92034,-0.198784
2.680427,2.65994,2.69127,2.704322,2.704322,-0.870833,-0.032979


In [51]:
#get data from polars
#labels=data['labels']
train_data=data['aapl_high']
#make polars data into torch tensors
labels=torch.tensor(labels).float()

train_data=torch.tensor(train_data).float()
train_data=train_data.to(device)

#fits labels to -5,5 and gives it 2 decimal places
labels=net_functions.fit_to_range_tensor(labels)
labels=torch.round(labels)
labels=labels.type(torch.LongTensor)
labels=labels.to(device)


In [52]:
#split data with sklearn
train_data,test_data,train_labels,test_labels=train_test_split(train_data,labels,test_size=.1)

#create pythorch datasets
train_data=torch.utils.data.TensorDataset(train_data,train_labels)
test_data=torch.utils.data.TensorDataset(test_data,test_labels)
batchsize=32
train_loader=DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader=DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [53]:
class TimeSeriesCNN(nn.Module):
    def __init__(self):
        super(TimeSeriesCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(64 * 7, 128)
        self.fc2 = nn.Linear(128, 1)  # Output size is 1 for regression, adjust for classification
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7)  # Adjust the view size based on the output size of the convolutional layers
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [54]:
# Function to train the CNN
def train_cnn(model, train_loader, num_epochs=10, learning_rate=0.001, device='cuda'):
    criterion = nn.MSELoss()  # Mean Squared Error loss for regression
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model.to(device)  # Move model to GPU
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            inputs = inputs.unsqueeze(1)  # Add channel dimension
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


In [55]:
model = TimeSeriesCNN()

In [56]:
train_cnn(model, train_loader, num_epochs=1000, learning_rate=0.001)

RuntimeError: Given groups=1, weight of size [32, 1, 3], expected input[1, 32, 1] to have 1 channels, but got 32 channels instead